## Лабораторна робота 4. **REGRESSION** 📈

# Опис вибірки "Motorbike Ambulance Calls"


Аварії за участю мотоциклів та відповідні виклики швидкої допомоги сильно залежать від екологічних та сезонних умов, таких як погодні умови, опади, день тижня, сезон, година дня тощо. Дані збирались протягом двох років щогодини та потім співвідносилися з відповідною погодою та сезонністю.

Основні характеристики вибірки `motorbike_ambulance_calls.csv`:

              - index: record index
              - date : date
              - season : season (1:springer, 2:summer, 3:fall, 4:winter)
              - yr : year (0: 2011, 1:2012)
              - mnth : month ( 1 to 12)
              - hr : hour (0 to 23)
              - holiday : whether day is holiday or not
              - weekday : day of the week
              - workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
              -  weathersit :
                      - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
                      - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
                      - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
                      - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
              - temp : Normalized temperature in Celsius. The values are divided to 41 (max)
              - atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
              - hum: Normalized humidity. The values are divided to 100 (max)
              - windspeed: Normalized wind speed. The values are divided to 67 (max)
              - cnt: count of total ambulance calls

## Імпорт необхідних бібліотек та завантаження даних

**Завдання 1** підготовка даних до опрацювання

1. завантажте вибірку `motorbike_ambulance_calls.csv`;   
2. підключіть необхідні бібліотеки;  
3. вивести основну статистичну інформацію по числовим змінним;  
4. окремо вивести максимальні та мінімальні значення по всім змінним;  
5. вивести розмірність датасету;

In [ ]:
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import pandas as pd

def get_data(data_path: str):
    data = pd.read_csv(data_path)
    return data

data = get_data('motorbike_ambulance_calls.csv')

print(data.describe())

print("Максимальні значення:", data.max())
print("Мінімальні значення:", data.min())
print("Розмірність датасету:", data.shape)


**Завдання 2** перевірити типи даних та перевірте кількість даних по кожній змінній.

In [ ]:
print("Типи даних:\n", data.dtypes)
print("\nКількість даних по кожній змінній:\n", data.count())

**Завдання 3** пепевірте всі змінні на наявність пропусків.

In [ ]:
print("Наявність пропусків у кожній змінній:\n", data.isnull().sum())

**Висновки з 1-3 завдання (детальні)**

Загалом, дані виглядають досить чистими, без пропущених значень, що полегшує їх подальший аналіз та обробку. Типи даних відповідають очікуваним типам для кожної змінної. Надалі можна переходити до більш детального аналізу даних та побудови моделей.



**Завдання 4** перетворіть змінну `date` в **date type**. Визначте часові межі датасету.

In [ ]:
from datetime import datetime

def tranform_date(df: pd.DataFrame, date_col_name: str):
    df[date_col_name] = pd.to_datetime(df[date_col_name])
    return df

data = tranform_date(data, 'date')

def get_time_limits(df: pd.DataFrame, date_col_name: str):
    first_day = df[date_col_name].min()
    last_day = df[date_col_name].max()
    print(f"Перший день датасету - {first_day}.\nОстанній день датасету - {last_day}")

get_time_limits(data, 'date')


**Висновок:** Було здійснено перетворення стовпця 'date' у тип даних 'date' за допомогою функції pd.to_datetime(). Це дозволяє коректно працювати з датами, забезпечуючи правильне їхнє відображення та порівняння.

Після цього були визначені часові межі датасету за допомогою визначення мінімальної та максимальної дати у стовпці 'date'. Це дозволило отримати перший та останній дні з датасету.

#Аналіз категоріальних змінних

**Завдання 5** виділити категоріальні змінні в окремий датасет







In [ ]:
data_categorical = data.select_dtypes(include=['object'])
print(data_categorical)

**Завдання 6** перетворіть змінну `season` в числову шкалу.

In [ ]:
def encode_season(df: pd.DataFrame, season_col_name: str):
    # Нічого не змінюємо, оскільки значення вже в числовій шкалі
    return df

data = encode_season(data, 'season')
print(data.head())

**Завдання 7** побудувати графіки `countplot` по всім категоріальним змінним

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 4, figsize=(20, 10))

for variable, subplot in zip(data_categorical.columns, ax.flatten()):
    sns.countplot(data_categorical[variable], ax=subplot)
    subplot.set_title(variable)

plt.tight_layout()
plt.show()


**Висновки з 5-7 завдання (детальні)**

1. Було створено окремий датасет data_categorical, який містить лише категоріальні змінні з основного датасету.
Для цього було використано метод select_dtypes() бібліотеки pandas, який дозволяє обирати стовпці за типом даних.

2. Змінна season була залишена без змін, оскільки її значення вже є у числовій шкалі.
Функція encode_season() була створена, але вона не змінювала значення змінної season.

3. Було побудовано графіки countplot для всіх категоріальних змінних у датасеті data_categorical.

#Аналіз числових змінних

**Завдання 8** виділити числові змінні в окремий датасет

In [ ]:
data_numerical = data.select_dtypes(include='number')
print(data_numerical)

**Завдання 9** побудуйте гістограми розподілу по всім числовим змінним

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 10))
data_numerical.hist(ax=ax)
plt.tight_layout()
plt.show()


**Завдання 10** побудуйте гістограму розподілу цільової змінної та розрахуйте коефіціент нахилу

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(data['cnt'], bins=30, color='skyblue', edgecolor='black')
plt.xlabel('Count of Ambulance Calls')
plt.ylabel('Frequency')
plt.title('Distribution of Count of Ambulance Calls')
plt.grid(True)
plt.show()

s = data['cnt'].skew()
print("Коефіцієнт нахилу:", s)

**Висновки з 8-10 завдання (детальні)**

1. Було створено окремий датасет data_numerical, який містить лише числові змінні з основного датасету.
2. Була побудована гістограма розподілу для кожної числової змінної у датасеті data_numerical.
Гістограми допомагають візуалізувати розподіл кожної змінної і отримати уявлення про її розподіл. 
3. Була побудована гістограма розподілу для цільової змінної cnt, яка представляє кількість викликів швидкої допомоги.
Розрахований коефіцієнт нахилу для цільової змінної. Значення коефіцієнта нахилу вказує на скошення розподілу цільової змінної: позитивне значення вказує на скошення вправо, а негативне - на скошення вліво.

#Аналіз взаємозв'язків між числовими і категоріальними змінними

**Завдання 11** побудуйте графіки співвідношення `scatterplot` між всіма числовими змінними і цільовою змінною `cnt`

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

for var, subplot in zip(data_numerical.drop('cnt', axis=1), ax.flatten()):
    sns.scatterplot(x=var, y='cnt', data=data_numerical, ax=subplot)

plt.tight_layout()
plt.show()

**Завдання 12** побудуйте графіки співвідношення `boxplot`[boxplot](https://matplotlib.org/stable/gallery/statistics/boxplot_demo.html) між категоріальними змінними і цільовою змінною `cnt`.

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 10))

for var, subplot in zip(data_categorical.columns, ax.flatten()):
    sns.boxplot(x=var, y='cnt', data=data, ax=subplot)

plt.tight_layout()
plt.show()

**Висновки з 11-12 завдання (детальні)**

1. Було побудовано графіки співвідношення між кожною з числових змінних і цільовою змінною cnt.
Графіки показують, як змінюється кількість викликів швидкої допомоги в залежності від значень числових змінних.
2. Було побудовано графіки співвідношення boxplot між кожною категоріальною змінною і цільовою змінною cnt.
Графіки boxplot дозволяють візуалізувати розподіл кількості викликів швидкої допомоги для кожної категорії категоріальної змінної, а також виявляти потенційні викиди (викиди) у даних. 

**Завдання 13** Побудувати матрицю кореляції:  

1.   Вирахувати кореляційні коефіціенти для набору данних за методом Пірсона
2.   Відобразити отриману матрицю коефіціентів у вигляді теплокарти  
3.   Вирахувати кореляційні коефіціенти для набору данних за методом Спірмана
4.   Відобразити отриману матрицю коефіціентів у вигляді теплокарти
5. Описати отримані результати, спираючись на значення коефіціентів кореляції та пояснити в чому відмінність обраних методів.

In [ ]:
pearson_corr = data_numerical.corr(method='pearson')

plt.figure(figsize=(10, 6))
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
plt.title('Pearson Correlation Matrix')
plt.show()

spearman_corr = data_numerical.corr(method='spearman')

plt.figure(figsize=(10, 6))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
plt.title('Spearman Correlation Matrix')
plt.show()


Після виконання завдання було отримано дві матриці кореляції: одна за методом Пірсона, а інша - за методом Спірмена.

**1. Метод Пірсона:**
Були виявлені як позитивні, так і негативні кореляційні зв'язки між числовими змінними.
Найбільші кореляції спостерігаються між показниками, що мають логічний зв'язок, наприклад, між температурою та відчуттям температури (temp і atemp).
Цей метод враховує тільки лінійні зв'язки між змінними, тому не може виявити нелинійні залежності.

**2. Метод Спірмена:**
Враховує порядок значень змінних, а не їхні конкретні значення, що робить його менш чутливим до викидів у даних та відсутність лінійності.
Якщо зв'язок між змінними не є лінійним, Спірмен може виявити цей зв'язок краще, ніж Пірсон.
Отримані кореляційні коефіцієнти Спірмена дають додаткове уявлення про зв'язки між змінними.

**Завдання 14** 
1. побудувати модель лінійної регресії:
2. видалити змінні, які НЕ є релевантними;

3. поділити вибірку на `train`, `test` та `validation` в пропорції `70/30`;  

4. провести навчання моделі лінійної регресії;

1. Для подальшої роботи з моделями для кожної категоріальної змінної ми створимо фіктивні змінні, щоб уникнути неправильного порядку категорій. [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

In [55]:
def get_dummies(df: pd.DataFrame):
    features = pd.concat([df,
                          pd.get_dummies(df['season'], prefix='season'),
                          ], axis=1)
    features = features.drop(['season', 'mnth', 'weekday', 'weathersit'], axis=1)
    return features
features = get_dummies(data)


In [56]:
features = get_dummies(data)

2. На основі змінної часу створіть нову змінну, яка відповідатиме за день і ніч.

In [57]:
# Визначення чи є година "ніччю" (1) чи "денною" (0)
features['night_hours'] = ((data['hr'] >= 22) | (data['hr'] <= 6)).astype(int)
features = features.drop('hr', axis=1)

**Модель лінійної регрессії  

[train_linear_model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Після отримання фіктивних змінних потрібно видалити по одній з кожної категорії, щоб запобігти мультиколінеарності. Також видаляємо ще два атрибути `date` та `atemp` - поясніть чому?

In [ ]:
features_lr = features.drop(['spring', 'mnth_4', 'weekday_0', 'weathersit_1', 'hr_0'], axis=1)

In [61]:
def get_train_data(df: pd.DataFrame, target: str, test_size: float):
    X = df.drop(target, axis=1)  # усі ознаки крім цільової змінної
    y = df[target]  # цільова змінна
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle=True)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_data(features_lr, 'cnt', 0.3)

In [ ]:
def train_linear_model(X_train, y_train):
  model = LinearRegression()
  model.fit(X_train, y_train)
  return model

model = train_linear_model(X_train, y_train)

In [ ]:
# predict
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
# scores
print('MSE train: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

Оскільки в нашій моделі є багато незалежних змінних, ми не можемо відобразити їх залежність у двовимірному просторі, але ми можемо побудувати графік взаємозв'язку між залишками моделі та передбачуваними значеннями, що також допоможе нам оцінити якість моделі.

In [ ]:
plt.scatter(y_train_pred,  y_train_pred - y_train,
            c='#5f93ad', marker='o', label='Training data')
plt.scatter(y_test_pred,  y_test_pred - y_test,
            c='#98c3d9', marker='s', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-220, xmax=1000, lw=2, color='#000000')
plt.tight_layout()

**Висновки з завдання (детальні)**



1.   
2.   
3. ...

